In [1]:
import tensorflow as tf
import numpy as np
import mnist
import matplotlib.pyplot as plt
import IPython.display as display
from utils import *

In [2]:
def create_model():
    model = tf.keras.Sequential(
    [
        tf.keras.layers.Reshape(input_shape=(28*28,), target_shape=(28, 28, 1)),
        
        tf.keras.layers.Conv2D(kernel_size=3, filters=12, use_bias=False, padding='same'),
        tf.keras.layers.BatchNormalization(center=True, scale=False),
        tf.keras.layers.Activation('relu'),
        
        tf.keras.layers.Conv2D(kernel_size=6, filters=24, use_bias=False, padding='same', strides=2),
        tf.keras.layers.BatchNormalization(center=True, scale=False),
        tf.keras.layers.Activation('relu'),
        
        tf.keras.layers.Conv2D(kernel_size=6, filters=32, use_bias=False, padding='same', strides=2),
        tf.keras.layers.BatchNormalization(center=True, scale=False),
        tf.keras.layers.Activation('relu'),
        
        tf.keras.layers.Flatten(),
        
        tf.keras.layers.Dense(200, use_bias=False),
        tf.keras.layers.BatchNormalization(center=True, scale=False),
        tf.keras.layers.Activation('relu'),
        
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

    model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.01),
                loss='categorical_crossentropy',
                metrics=['accuracy'])
    return model
model = create_model()
# print model layers
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape (Reshape)           (None, 28, 28, 1)         0         
                                                                 
 conv2d (Conv2D)             (None, 28, 28, 12)        108       
                                                                 
 batch_normalization (BatchN  (None, 28, 28, 12)       36        
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 28, 28, 12)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 24)        10368     
                                                                 
 batch_normalization_1 (Batc  (None, 14, 14, 24)       72        
 hNormalization)                                        

# Using our own dataset

In [4]:
from sklearn.model_selection import train_test_split
X,y = get_data()
# X = X.reshape(-1, 28*28)
train_digit,test_digit, train_label, test_label = train_test_split(X,y, test_size=0.1, random_state=42)
[tmp.shape for tmp in [train_digit, train_label,test_digit, test_label]]

[(992, 784), (992,), (111, 784), (111,)]

In [5]:
train_label = tf.one_hot(train_label,10)
test_label = tf.one_hot(test_label,10)

TRAIN_SIZE = len(train_digit)
EPOCHS = 10
steps_per_epoch = TRAIN_SIZE//EPOCHS
BATCH_SIZE  = TRAIN_SIZE//steps_per_epoch

print("Steps per epoch: ", steps_per_epoch)
history = model.fit(x = train_digit, y = train_label , steps_per_epoch=steps_per_epoch, epochs=EPOCHS,
                    validation_data=(test_digit, test_label), validation_steps=1)
model.save('saved_model/local_model')

Steps per epoch:  99
Epoch 1/10
99/99 [==============================] - 4s 24ms/step - loss: 0.8594 - accuracy: 0.7407 - val_loss: 0.6708 - val_accuracy: 0.8018
Epoch 2/10
99/99 [==============================] - 2s 21ms/step - loss: 0.2558 - accuracy: 0.9333 - val_loss: 0.3987 - val_accuracy: 0.8829
Epoch 3/10
99/99 [==============================] - 2s 21ms/step - loss: 0.1837 - accuracy: 0.9611 - val_loss: 0.2297 - val_accuracy: 0.9279
Epoch 4/10
99/99 [==============================] - 2s 21ms/step - loss: 0.1380 - accuracy: 0.9667 - val_loss: 0.1769 - val_accuracy: 0.9279
Epoch 5/10
99/99 [==============================] - 2s 21ms/step - loss: 0.0939 - accuracy: 0.9796 - val_loss: 0.1509 - val_accuracy: 0.9369
Epoch 6/10
99/99 [==============================] - 2s 21ms/step - loss: 0.0844 - accuracy: 0.9815 - val_loss: 0.2305 - val_accuracy: 0.9369
Epoch 7/10
99/99 [==============================] - 2s 21ms/step - loss: 0.0647 - accuracy: 0.9898 - val_loss: 0.1776 - val_accuracy:

99/99 [==============================] - 2s 19ms/step - loss: 0.0388 - accuracy: 0.9950 - val_loss: 0.1363 - val_accuracy: 0.9640


# MNIST

In [14]:
RUN_MNIST = False
if RUN_MNIST:
    model = create_model()
    train_digit, train_label = mnist.extract_train()
    test_digit, test_label = mnist.extract_test()
    train_digit = train_digit.reshape(-1,28*28)
    test_digit = test_digit.reshape(-1,28*28)

    train_label = tf.one_hot(train_label, 10)
    test_label = tf.one_hot(test_label, 10)
    print([tmp.shape for tmp in [train_digit, train_label, test_digit, test_label]])

    TRAIN_SIZE = len(train_digit)
    BATCH_SIZE = 128
    EPOCHS = 10
    steps_per_epoch = TRAIN_SIZE//BATCH_SIZE  # 60,000 items in this dataset


    print("Steps per epoch: ", steps_per_epoch)
    history = model.fit(x = train_digit, y = train_label , steps_per_epoch=steps_per_epoch, 
                        epochs=EPOCHS,
                        validation_data=(test_digit, test_label), validation_steps=1)
    model.save('saved_model/googler_model')

[(60000, 784), TensorShape([60000, 10]), (10000, 784), TensorShape([10000, 10])]
Steps per epoch:  468
Epoch 1/10
468/468 [==============================] - 59s 123ms/step - loss: 0.1438 - accuracy: 0.9583 - val_loss: 0.0577 - val_accuracy: 0.9809
Epoch 2/10
468/468 [==============================] - 54s 115ms/step - loss: 0.0424 - accuracy: 0.9875 - val_loss: 0.1797 - val_accuracy: 0.9464
Epoch 3/10
468/468 [==============================] - 52s 110ms/step - loss: 0.0295 - accuracy: 0.9915 - val_loss: 0.0363 - val_accuracy: 0.9883
Epoch 4/10
468/468 [==============================] - 54s 115ms/step - loss: 0.0228 - accuracy: 0.9929 - val_loss: 0.0257 - val_accuracy: 0.9908
Epoch 5/10
468/468 [==============================] - 54s 116ms/step - loss: 0.0168 - accuracy: 0.9949 - val_loss: 0.0319 - val_accuracy: 0.9890
Epoch 6/10
468/468 [==============================] - 54s 116ms/step - loss: 0.0136 - accuracy: 0.9956 - val_loss: 0.0441 - val_accuracy: 0.9873
Epoch 7/10
468/468 [=======

468/468 [==============================] - 52s 110ms/step - loss: 0.0074 - accuracy: 0.9979 - val_loss: 0.0253 - val_accuracy: 0.9913


In [24]:
digits, labels = get_data()
digits = digits.reshape(-1,28*28)

pred = model.predict(digits)
pred = [np.argmax(p) for p in pred]
pred = np.array(pred)

res = pred == labels
acc = res[res].sum() / res.shape[0] * 100
acc

35/35 [==============================] - 0s 8ms/step


88.21396192203082

In [25]:
digits, labels = mnist.extract_test()
# digits, labels = get_data()
digits = digits.reshape(-1,28*28)

pred = model.predict(digits)
pred = [np.argmax(p) for p in pred]
pred = np.array(pred)

res = pred == labels
acc = res[res].sum() / res.shape[0] * 100
acc

313/313 [==============================] - 3s 8ms/step


99.13